In [1]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


#감정분석

####환경설정

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661776 sha256=b79c791f98fc53e024b6d3bb17712efed79c6918844f051c7cd362c3f9f3e2b6
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-88my0gk_/kobert-tokenizer_90cf8cea74854fe6bb013c449aad6356
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-88my0gk_/kobert-tokenizer_90cf8cea74854fe6bb013c449aad6356
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=b1ab66d3d69b54a62ae264140ece22b6d570838efc74b350f960423a2f58d24e
  Stored in directory: /tmp/pip-ephem-wheel-cache-06snl5jr/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
device = torch.device("cuda:0")

In [6]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

####데이터 전처리

In [8]:
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = '/content/drive/MyDrive/2023_ML_proj/최종/emotion_data_final.csv'

# CSV 파일을 DataFrame으로 로드
df = pd.read_csv(csv_file_path)

# DataFrame 내용 확인
data = df

data

,emotion,sentence
0,화,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,화,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,화,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,화,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,화,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
46416,슬픔,나이가 다 돼서 퇴직을 하게 되었어. 하지만 노후 준비를 제대로 안 해서 돈 걱정이...
46417,스트레스,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
46418,걱정,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
46419,걱정,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [9]:
data.loc[(data['emotion'] == "화"), 'emotion'] = 0
data.loc[(data['emotion'] == "편안"), 'emotion'] = 1
data.loc[(data['emotion'] == "걱정"), 'emotion'] = 2
data.loc[(data['emotion'] == "당황"), 'emotion'] = 3
data.loc[(data['emotion'] == "만족"), 'emotion'] = 4
data.loc[(data['emotion'] == "괴로움"), 'emotion'] = 5
data.loc[(data['emotion'] == "후회"), 'emotion'] = 6
data.loc[(data['emotion'] == "슬픔"), 'emotion'] = 7
data.loc[(data['emotion'] == "스트레스"), 'emotion'] = 8
data.loc[(data['emotion'] == "신뢰"), 'emotion'] = 9
data.loc[(data['emotion'] == "신남"), 'emotion'] = 10
data.loc[(data['emotion'] == "실망"), 'emotion'] = 11
data.loc[(data['emotion'] == "안도"), 'emotion'] = 12
data.loc[(data['emotion'] == "억울"), 'emotion'] = 13
data.loc[(data['emotion'] == "열등감"), 'emotion'] = 14
data.loc[(data['emotion'] == "회의적"), 'emotion'] = 15
data.loc[(data['emotion'] == "외로운"), 'emotion'] = 16
data.loc[(data['emotion'] == "혐오"), 'emotion'] = 17
data.loc[(data['emotion'] == "자신감"), 'emotion'] = 18
data.loc[(data['emotion'] == "기쁨"), 'emotion'] = 19
data.loc[(data['emotion'] == "충격"), 'emotion'] = 20
data.loc[(data['emotion'] == "질투"), 'emotion'] = 21
data.loc[(data['emotion'] == "감사"), 'emotion'] = 22

data

,emotion,sentence
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,0,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,0,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
46416,7,나이가 다 돼서 퇴직을 하게 되었어. 하지만 노후 준비를 제대로 안 해서 돈 걱정이...
46417,8,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
46418,2,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
46419,2,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [10]:
# 필요한 데이터를 data_list 배열에 저장
data_list = []
for sentence, label in zip(data['sentence'], data['emotion']):
    data = []
    data.append(sentence)
    data.append(str(label))
    data_list.append(data)

for data in data_list:
    print(data)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['상사가 나한테 잘해주니 시샘하는 사람이 있네. 자꾸 의식해서 업무에 집중할 수가 없어. 그런 사람을 무시하고 싶은데 쉽지 않아.', '14']
['큰애가 드디어 초경을 해. 근데 초경을 부끄러워해. 축하할 일이라고 말해 줘야지.', '2']
['새로운 일을 맡다 보니 어렵네. 그런가 봐. 힘들어서 그만하고 싶어. 없어. 당장은 힘들어서 그만하고 싶을 뿐이야.', '2']
['부장님이 업무지시를 한 번에 했으면 좋겠는데 자꾸 호출하셔서 엄청 성가셔. 왔다 갔다 하는 것도 그런데 일하는 중에 부르셔서 자꾸 흐름이 깨지니까 일 한 건 처리하는데도 시간이 더 걸리네. 몇 번 말한 적이 있는데 고쳐지질 않아.', '8']
['얼마 전에 건강검진을 받았는데 자궁에 혹이 생겼다고 그래서 당황스럽더라. 전에 혹이 있어서 수술한 지 오래되지 않았거든. 그런 사람이 있다고는 하던데 그게 나일 줄은 몰랐네. 다행인 거는 폐경이 다가오면 혹이 많이 커지지는 않는다고 하더라고. 괜찮다고 하더라고.', '3']
['갑자기 명예 퇴직 발표 됐어. 은퇴 준비를 해야 할 거 같아. 그렇지만 어떻게 하겠어. 일은 저질러 졌는데. 없어. 지금부터라도 세워야 하는데 막막하다.', '2']
['돈 들어갈 곳은 많은데 코로나 때문에 회사 재정이 좋지 않아서 은퇴 준비를 해야겠어. 아직 노후 준비도 안 돼 있는데 회사가 어려워지니 계속 다닐 수도 없을 노릇이야. 없지. 지금부터 세워야 하는데 막막하네.', '2']
['점점 갱년기 증상도 심해지고 호르몬 변화도 생기다 보니 몸이 예전 같지 않아 걱정이야. 그동안 약도 많이 챙겨 먹고 꾸준히 운동도 해서 괜찮을 줄 알았는데 나이는 못 속이네. 노력도 나이를 이기지 못하는 거 같아.', '2']
['나에게 은퇴라는 말은 먼 일이라고 생각하고 너무 즐기기만 한 거 같아 화가 나. 요즘은 노후 준비도 해야 하니 재정이 여유롭지 않으면 살기가 점점 더 어려워지는 세상인 거 같아. 조금 있는 재

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['늦은 나이에 결혼을 하려는데 경제적으로 힘드니 무기력해져. 노후준비도 안 된 상황에서 결혼비용까지 드니 결혼을 해야 할지도 아직까지 고민돼. 자녀들과 부인 될 사람에게 솔직하게 고민을 말해보려 해.', '15']
['건강이 나빠지다 보니 남편을 뒷바라지하는데 힘이 너무 부쳐. 나 자신이 싫어져. 집 앞 마트에 장을 보러 나가는 것도 힘드니까 스스로가 바보가 되는 기분이야. 남편에게 솔직하게 내 건강 상태를 말하고 배려를 구할 생각이야.', '17']
['나이가 들고 소득이 없으니 자꾸 만나는 사람이 줄어서 고독해지고 허탈감이 들어. 예전에 잘 살았을 때는 먼저 다가오더니 힘드니까 내 곁을 다 떠나는 것 같아서 슬퍼. 내 자신의 내면을 수양해서 상처받지 않게 노력해보려 해.', '7']
['내 친구랑 갈등이 생겼는데 부모님이 내 잘못도 있다고 하셨어. 처음엔 억울했는데 다시 생각하니까 정말 내가 잘못한 것 같아서 죄책감이 느껴져. 내가 잘못한 것은 사과하고 친구랑 화해해야겠어.', '14']
['요즘에 진로를 정하는데 너무 고민이 돼. 내가 하고 싶은 것을 이루지 못할 것 같아. 나를 괴롭히는 친구들이 나의 꿈을 비아냥거리면서 괴롭혀. 친구들 말은 신경 안 쓰고 열심히 노력해서 내 꿈을 꼭 이룰 거야.', '11']
['우리 집은 재혼 가정인데 나는 몸에 질환이 있고 그래서 엄마가 더 힘들어해. 근데 새아빠가 우리에게 무관심해서 내가 나가서 일하면서 생활비를 보태고 있어. 아무래도 같이 다 모여서 이야기를 좀 해봐야겠어.', '6']
['엄마랑 아빠가 자꾸 내가 하는 것마다 반대하면서 손찌검을 해. 나 너무 우울해. 너무 힘들어. 내 엄마 아빠 같지 않아. 학교나 주변 어른들에게 알려야겠어. 그리고 나부터 상담 받고 다시 건강하게 지내고 싶어.', '11']
['우리 학교 친구 중에 어떤 애가 왕따를 당하는데 너무 안쓰러워. 거기다가 걔는 신체장애까지 있어서 더 힘들어하더라고. 일단 내가 나서는 거는 위험할 수도 있으니 선생님이나 경찰에게 알려서 해결하도록 

In [11]:
# train data와 test data로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=32)

print(len(dataset_train)) # 34815 출력
print(len(dataset_test)) # 11606 출력

37136
9285


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

####파라미터 세팅

In [13]:
# Setting parameters
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


####Bert Classification 모델 정의

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=23,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
import os
import torch

In [22]:
from transformers import BertForSequenceClassification, BertTokenizer

# Hugging Face의 BERT 모델과 토크나이저 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
todel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# 저장된 모델 가중치 불러오기
saved_model_path = "/content/drive/MyDrive/2023_ML_proj/최종/KoBERT 학습 모델/final_model_epoch10.pt"
model.load_state_dict(torch.load(saved_model_path))

# 모델을 evaluation mode로 설정 (필요에 따라)
model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [23]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another,  0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            # Get indices of top 3 predictions
            top3_indices = np.argsort(logits)[-3:][::-1]

            top3_predictions = []

            for idx in top3_indices:
                if idx == 0:
                  top3_predictions.append("화가")
                elif idx == 1:
                  top3_predictions.append("편안함이")
                elif idx == 2:
                  top3_predictions.append("걱정이")
                elif idx == 3:
                  top3_predictions.append("당황이")
                elif idx == 4:
                  top3_predictions.append("만족함이")
                elif idx == 5:
                  top3_predictions.append("괴로움이")
                elif idx == 6:
                  top3_predictions.append("후회가")
                elif idx == 7:
                  top3_predictions.append("슬픔이")
                elif idx == 8:
                  top3_predictions.append("스트레스가")
                elif idx == 9:
                  top3_predictions.append("신뢰가")
                elif idx == 10:
                  top3_predictions.append("신남이")
                elif idx == 11:
                  top3_predictions.append("실망이")
                elif idx == 12:
                  top3_predictions.append("안도가")
                elif idx == 13:
                  top3_predictions.append("억울함이")
                elif idx == 14:
                  top3_predictions.append("열등감이")
                elif idx == 15:
                  top3_predictions.append("회의감이")
                elif idx == 16:
                  top3_predictions.append("외로움이")
                elif idx == 17:
                  top3_predictions.append("혐오가")
                elif idx == 18:
                  top3_predictions.append("자신감이")
                elif idx == 19:
                  top3_predictions.append("기쁨이")
                elif idx == 20:
                  top3_predictions.append("충격이")
                elif idx == 21:
                  top3_predictions.append("질투가")
                elif idx == 22:
                  top3_predictions.append("감사가")

            test_eval.append(top3_predictions)
        return test_eval

In [24]:
sentence = input("하고싶은 말을 입력해주세요 : ")
predictions_top3 = predict(sentence)
print(predictions_top3)

하고싶은 말을 입력해주세요 : 나 너무 우울하고 힘들어


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[['슬픔이', '괴로움이', '후회가']]


#cos 유사도

In [25]:
!pip install pandas scikit-learn

In [26]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 36.6 MB/s eta 0:00:00


In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
# CSV 파일에서 데이터 로드
csv_file_path = '/content/drive/MyDrive/2023_ML_proj/최종/도서데이터.csv'
df = pd.read_csv(csv_file_path)

# NaN 값이 있는 경우 처리
df['Tokenized'].fillna('', inplace=True)

# 사용할 형태소 분석기 (예: KoNLPy의 Okt)
from konlpy.tag import Okt
okt = Okt()

# 형태소 분석 및 불용어 처리 함수 정의
def tokenize(text):
    tokens = okt.nouns(text)
    return tokens

# TfidfVectorizer를 사용하여 텍스트를 벡터화
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=[], use_idf=True)
tfidf_matrix = vectorizer.fit_transform(df['Tokenized'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [30]:
# 입력 문장을 토큰화하여 벡터화하는 함수 정의
def vectorize_input(input_text):
    input_vector = vectorizer.transform([input_text])
    return input_vector

In [31]:
# 코사인 유사도 계산 함수 정의
def calculate_similarity(input_vector, tfidf_matrix):
    cosine_similarities = cosine_similarity(input_vector, tfidf_matrix)
    return cosine_similarities

In [32]:
# 책 추천 함수 정의
def recommend_book(input_text, df, tfidf_matrix):
    input_vector = vectorize_input(input_text)
    cosine_similarities = calculate_similarity(input_vector, tfidf_matrix)

    # 가장 유사한 책 찾기
    similar_books = list(enumerate(cosine_similarities[0]))
    sorted_books = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:]

    # 상위 5개 추천
    top_recommendations = sorted_books[:5]

    # 추천 도서 정보 출력
    for i, score in top_recommendations:
        print(f"Book ID: {i}, Similarity Score: {score}")
        print(f"Title: {df.iloc[i]['제목']}")
        print("\n")


In [33]:
##추출한 감정 처리
# 리스트의 각 원소를 공백으로 구분된 하나의 문자열로 합치기
query = ' '.join(map(str, predictions_top3))
# 입력 데이터도 TF-IDF 벡터화
query_vector = vectorizer.transform([query])
# 코사인 유사도 계산
cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

# 상위 5개 유사도 및 인덱스 찾기
top5_indices = cosine_similarities.argsort()[-5:][::-1]
top5_titles = df.iloc[top5_indices]['제목'].tolist()
top5_similarity_scores = cosine_similarities[top5_indices]

# 결과 출력
print("유사도 순으로 상위 5개 책:")
for i, (title, score) in enumerate(zip(top5_titles, top5_similarity_scores), 1):
    print(f"{i}. 책 제목: {title}, 유사도: {score}")

# 최고 유사도를 가지는 책의 인덱스 및 제목 출력
max_similarity_index = cosine_similarities.argmax()
result_title = df.iloc[max_similarity_index]['제목']
max_similarity_score = cosine_similarities[max_similarity_index]

유사도 순으로 상위 5개 책:
1. 책 제목: 끌리는 말투 호감 가는 말투, 유사도: 0.12147958792266883
2. 책 제목: 게으른 뇌에 행동 스위치를 켜라, 유사도: 0.10175032218090929
3. 책 제목: 세상의 마지막 기차역(리커버 에디션), 유사도: 0.09872790129528755
4. 책 제목: 기분이 태도가 되지 않게, 유사도: 0.08679157247168783
5. 책 제목: 일의 기쁨과 슬픔, 유사도: 0.0841859907279745


In [34]:
input_text = sentence
recommend_book(input_text, df, tfidf_matrix)


Book ID: 154, Similarity Score: 0.3409689325026031
Title: 그렇게 말해줘서 고마워


Book ID: 217, Similarity Score: 0.2561938188060647
Title: 나에게 고맙다(30만 부 기념 전면 개정판)


Book ID: 306, Similarity Score: 0.233864850240888
Title: 모든 순간이 너였다


Book ID: 157, Similarity Score: 0.16682001956802817
Title: 괜찮은 척 말고, 애쓰지도 말고


Book ID: 223, Similarity Score: 0.16371042951441123
Title: 나로서 충분히 괜찮은 사람


